In [1]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

In [3]:
from qdrant_client import QdrantClient, models

client = QdrantClient(url="http://localhost:6333")

client.create_collection(
    collection_name="test",
    vectors_config=models.VectorParams(size=4096, distance=models.Distance.COSINE),
)

# client.delete_collection(collection_name="test")

client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='test')])